## Подготовка данных
Тут собирается очень сложно. Должен быть один файл с данными по клиенту и флагом дефолта по нему

In [ ]:
#путь к файлу
strategy_data_path='C:\\Временные файлы\\strategy_data.csv'

In [ ]:
#import всего нужного 
import os
#читаем файл с данными по клиентам
pwd = os.getcwd()
os.chdir(os.path.dirname(strategy_data_path))
strategy_data=pd.read_csv(os.path.basename(strategy_data_path), delimiter=';', decimal='.',quotechar="'", encoding='1251')
os.chdir(pwd)

In [ ]:
#читаем и пробразуем файл с инфой по дефолтам
strategy_result_path='C:\\Users\\Admin\\Desktop\\strategy_result.csv'
strategy_result=pd.read_csv(strategy_result_path, delimiter=';', decimal='.',quotechar="'", encoding='1251')
strategy_result.rename(index=str, columns={"ApplicationID": "ApplicationId", "target_90p_12mob_objective": "IsDefault"},inplace=True)
strategy_result.reset_index(inplace=True)
strategy_result=strategy_result[['ApplicationId', 'IsDefault']]

In [ ]:
#Данные по клиентам с признаком дефолта (НО тк в предоставленных данных айди у заявок разные, джоиним рандомно)
#full_data=strategy_data.join(other=strategy_result, on='ApplicationId', how='left', rsuffix='_r')
full_data=strategy_data.join(other=strategy_result, rsuffix='_r')

## Анализ данных по клиентам

In [ ]:
%run -i datapy/analyze/recomendation.py

server=recommender()

In [ ]:
#Набор полей, по которым ищутся правила, передаётся клиентом
columns=['ClientIncomeAddit6','HPoint_MinValue','ClientJobSalary6','ClientExpenseTotal']
#Поле - флаг дефолта, передаётся клиентом
default_column='IsDefault'
result=server.RecommendRules(full_data, columns, default_column)
result

### Пример вывода данных по предлагаемым правилам пользователю

In [ ]:
#пример обработк полученных правил
def RuleToText(rule):
    column_name=rule[0]
    thr=rule[1]
    trend=rule[2]
    if trend is None:
        return "Для поля {} не было найдено подходящих правил".format(column_name)
    elif trend:
        return "Рекомендуется ограничить выдачи по тем заявкам, где {} выше {}".format(column_name, thr)
    else:
        return "Рекомендуется ограничить выдачи по тем заявкам, где {} ниже {}".format(column_name, thr)
    
for rule in result:
    print(RuleToText(rule))